In [11]:
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('E:/text_mining/dict.txt')  #切換至中文繁體字庫
jieba.load_userdict('E:/text_mining/dict_keyw.txt')  #
jieba.load_userdict('E:/text_mining/dict_cbdic.txt')  #
#預設就是自己
client = MongoClient('10.120.28.14',27017)  #不用修改  字典檔請改位子
#client = MongoClient('localhost:27017')  #不用修改  字典檔請改位子

database = client['news']
collection =database['news']

Building prefix dict from E:\text_mining\dict.txt ...
DEBUG:jieba:Building prefix dict from E:\text_mining\dict.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
DEBUG:jieba:Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
Loading model cost 0.896 seconds.
DEBUG:jieba:Loading model cost 0.896 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [50]:
for aa in range(1,4):
    print aa,"月開始"
    e=0
    for ii in range(0,4):        
        a=collection.find(
                {"$and":[                   
                        {"comp":{"$regex":"udn"}},
                        {"date":{"$regex":"20160{}{}".format(aa,ii)} }          #計算當月文章數
                        ]},{"_id":0}).count()  
        if a==0:
            print "---------------------------------------"
            print "20160{}{}".format(aa,ii),"count is 0"
            print "---------------------------------------"

        else:
            e=e+a
            print a
    print aa,"月,共有",e,"篇"

1 月開始
10864
13155
13180
1866
1 月,共有 39065 篇
2 月開始
10289
10384
11712
2016023 count is 0
2 月,共有 32385 篇
3 月開始
12529
13157
12733
3073
3 月,共有 41492 篇


In [48]:
a=collection.find(
                {"$and":[                   
                        {"comp":{"$regex":"udn"}},
                        {"date":{"$regex":"2016023" }}        #計算當月文章數
                        ]},{"_id":0}).count()

print a

0


In [47]:
 content =[]    
for post in collection.find(
    {"$and":[                   
            {"comp":{"$regex":"udn"}},                              #尋找一個月新聞(報社手動修改)
            {"date":{"$regex":"2016023"}}        #10-12月請將0去掉
            ]}): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray()
features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  

top_features = []
for n in range(0,a):  #迴圈參考上面的總文章數
    indices = np.argsort(weight[n])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
    # 看TOP多少的詞
    top_n = 40
    top_features.append([features[i] for i in indices[:top_n]])    

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [44]:
a=collection.find(
                {"$and":[                   
                        {"comp":{"$regex":"udn"}},
                        {"date":{"$regex":"2016023" }}        #計算當月文章數
                        ]},{"_id":0}).count()
if a<0:
    print a
else:
    print "nono"

nono


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import time
for aa in range(1,4):
    for ii in range (1,4):   # 1-9月    10月後改為10-13再執行
        a=collection.find(
            {"$and":[                   
                    {"comp":{"$regex":"udn"}},
                    {"date":{:"20160{}{}".format(aa,ii)}}           #計算當月文章數
                    ]},{"_id":0}).count()    
        content =[]
        tStart=time.time() #計算所需時間(計時開始)
        if a ==0:
            print "20160{}{}".format(aa,ii)
        else:
            for post in collection.find(
                {"$and":[                   
                        {"comp":{"$regex":"udn"}},
                        {"date":{:"20160{}{}".format(aa,ii)}}           #計算當月文章數
                        ]},{"_id":0})
                summary = post['content']
                content.append('/'.join(jieba.cut(summary)))

            vectorizer = TfidfVectorizer()
            X = vectorizer.fit_transform(content)  # titile 放文本
            weight = X.toarray()
            features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  

            top_features = []
            for n in range(0,a):  #迴圈參考上面的總文章數
                indices = np.argsort(weight[n])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
                # 看TOP多少的詞
                top_n = 40
                top_features.append([features[i] for i in indices[:top_n]])    #這邊的寫法會讓關鍵字中間會有空白     

            obj=collection.find(
                 {"$and":[                   
                        {"comp":{"$regex":"udn"}},
                        {"date":{:"20160{}{}".format(aa,ii)}}           #計算當月文章數
                        ]},{"_id":0})
            for i in range(0,a):  #迴圈參考上面的總文章數
                collection.update({"_id":obj[i]["_id"]},{"$set":{"tfidf":top_features[i]}}) #collection.update即可

            tEnd = time.time() #記時(時間結束)
            print '完成',len(top_features),'筆'    

            del content[:]  #清除list暫存記憶體
            del top_features[:]  #清除list暫存記憶體

            print ' 花了 {} s'.format(tEnd-tStart)    #將經過時間print 出來